# Signal-to-Noise Estimator for MU69 Occultation Events

Eliot Young, *SWRI*, 9-NOV-2016. Edited by HBT 10-Nov-2016 to fix some broken imports, etc.

In [11]:
from IPython.core.display import HTML
HTML("<style>.container { width:98% !important; }</style>")

**BASIC IDEA**

This notebook estimates the signal-to-noise ratio by estimating the flux of photons from the occulted star (the "signal") and the amplitudes of four noise sources: photon shot noise from the star, from sky background and from dark current, plus read noise from the detector. These four noise sources are added quadratically.

SNR = Source_Counts / sqrt(source_variance + sky_variance + dark_current_variance + readnoise^2)

We'll work in units of electrons. This means that the noise in the source signal is square root of the source counts (in electrons) or, alternatively, the variance in the signal is the square of the error, or the source counts itself -- that's the way that Poisson-distributed random variables (like collected photons) work.

Example: If an observation of a star results in 2500 electrons over the exposure, then the SNR of that measurement (ignoring sky counts, dark current and read noise) is 

SNR = 2500/sqrt(2500) = 50.



In [30]:
import math
import numpy as np

### Some Useful Constants

In [12]:
h = 6.62606e-34 # Planck constant in J·s
c = 2.998e8 # speed of light in m/s

f_R = 1.74e-9 * 1.0e-7 # Flux from a R=0 source in J cm-2 s-1 Å-1
lam_R = 0.7 # Center of the Johnson R filter (µm)
wid_R = 0.22 * 10000 # Width of the R filter in Å.
pi = 3.14

# Ref: Zombeck 2nd ed. p. 100 (below)

![Standard Photometric Systems: Zombeck 2nd ed., p. 100](Zombeck_p100.png "Standard Photometric Filters")

### Some useful routines

In [13]:
def fluxRatio(m1,m2):
    '''This routine returns the flux ratio of source 1 over source 2
    EFY, SWRI, 9-NOV-2016
    
    Inputs: 
    m1 - magnitude of source 1
    m2 - magnitude of source 2
    
    Output:
    ratio of flux1/flux2
    
    Example: fluxRatio(5.0, 0.0)
    > 0.01
    '''
    
    return(10.0**(0.4 * (m2 - m1)))

### Some Telescope and Detector Parameters

Assume that the telescope is a 20-in Newtonian with a small secondary flat (a 1% obscuration). Its speed is f/3.7. The average star PSF is assumed to be 2" wide (FWHM). Assume the telescope transmission is 88%.


Assume the detector is an sCMOS device (like the Andor/Zyla 4.2) with 6.5 µm pixels. QE is 82% in R-band and the readnoise is 1 e-.

In [14]:
aperture = 50.0 # Aperture in cm
telTrans = 0.88
clear_ap = 0.99
eff_ap = pi * (0.5*aperture)**2 * clear_ap * telTrans # effective aperture in cm^2

fnum = 3.7
focal_L = aperture * 10000. * fnum # Focal length in microns

pixSz = 6.5 # Detector pixel pitch in microns (Zyla)
QE = 0.82
RN = 1.0 # read noise, in e- per pixel (Zyla)
DC = 0.14 # dark current, in e- per pixel per sec at cold temperatures (Zyla)

PSF_wid = 2.0 # PSF radius of reasonable signal in arcsec, assuming mediocre seeing

platescale = (206265./focal_L) * pixSz # platescale in arcsec per pixel

sky_R = 19.9 # Sky is darker than this (R-mag per square arcsec) 50% of the time.
# Ref: <https://www.gemini.edu/sciops/telescopes-and-sites/observing-condition-constraints/optical-sky-background>

print("platescale (arcsec/pixel): ", platescale)
print("Effective telescope area (cm^2): ", eff_ap)

platescale (arcsec/pixel):  0.7247148648648649
Effective telescope area (cm^2):  1709.73


In [15]:
nPix = pi * (PSF_wid/platescale)**2 # Number of pixels covered by a star.
if (nPix < 1.0): nPix = 1
print("nPix: ", nPix)

nPix:  23.914169364287176


### Calculate the Flux from a Star

In [16]:
def StarPhots(m):
    '''This routine calculates the number of photons per sec received from our telescope from a star of R-mag = m.
    EFY, SWRI, 9-NOV-2016
    
    Inputs:
    m - the R-mag of a star
    
    Outputs:
    f - the flux in photons per sec per cm^2'''
    
    # Step 1: Calculate the flux from the star (J/s) in the R filter with our telescope
    
    fluxJ = f_R * wid_R * eff_ap
    
    # Step 2: Calculate photons per Joule and convert flux to phots/s
    
    freq = c/(lam_R * 1e-6) # freq of an R-band photon in Hz
    PhotJ = h * freq # J/phot
    
    fluxPhot = fluxJ/PhotJ
    
    return(fluxRatio(m, 0.0) * fluxPhot)
    

In [17]:
# Here's a convenience function to get the number of source electrons in a exposure of time t sec.
def nElec(m, t):
    '''This routine scales the photon flux by the QE to convert to electrons.
    
    Eliot Young, SWRI, 9-NOV-2016
    
    Inputs:
    m - source R-mag
    t - exposure time
    
    Outputs:
    nE - number of detected electrons from source in a t-sec exposure
    '''
    
    return(QE * StarPhots(m) * t)

In [18]:
nElec(15.4, 0.2)

261.67025441178816

### Estimate Noise Sources

In [19]:
# At a minimum, the noise will be limited by the Poisson noise of the source itself. 
# If we're lucky, that will be the dominant noise source
print("Best case SNR from an R=15.4 in 0.2 s (just considering photon shot noise from the source: ", sqrt(nElec(15.4, 0.2)))

NameError: name 'sqrt' is not defined

In [31]:
# A convenience function to quadratically add the noise sources
def NoiseTerms(m, t, nP):
    '''A function to estimate each of the four noise sources.
    
    EFY, SWRI, 9-NOV-2016
    
    Inputs:
    m - R-band magnitude
    t - exposure time (s)
    nP - estimated number of pixels covered by the star
    
    Outputs:
    nz - a four element array with the 4 noise terms: source_shot_noise, sky_shot_noise, dark_cur_shot_noise, readnoise
    
    Note: this function refers to a lot of globals that are defined in the cell that describes the telescope & detector'''
    
    # Source shot noise:
    src_nz = math.sqrt(nElec(m,t))
    
    # Sky shot noise -- we need to estimate the sky area under the PSF (in arcsec)
    pixArcSec = platescale**2 # area of a pixel in sq arcsec
    sky_nz = math.sqrt(nP * pixArcSec * nElec(sky_R, t))
    
    # Dark Current noise
    dark_nz = math.sqrt(DC * nP * t)
    
    NzTerms = np.array([src_nz, sky_nz, dark_nz, RN*nP])
    
    return(NzTerms)
    
    

In [32]:
NoiseTerms(15.4, 0.2, nPix) # Source counts and read noise are dominant

array([ 16.17622497,   7.21725414,   0.81828891,  23.91416936])

In [35]:
# SNR: a function to estimate the SNR of a star in a given exposure
def SNR_fun(m, t, nP):
    '''A function to calculate the ratio of the source counts to the quadratic sum of noise terms.
    
    EFY, SWRI, 9-NOV-2016
    
    Inputs:
    m - R-band magnitude
    t - exposure time (s)
    nP - estimated number of pixels covered by the star
    
    Outputs:
    SNR - an estimate of the SNR
    '''
    
    src_e = nElec(m, t)
    nz = NoiseTerms(m, t, nP)
    
    return(src_e/math.sqrt(sum(nz**2)))
    
    

In [36]:
SNR_fun(15.4, 0.2, nPix)

8.789416330459927

In [37]:
SNR_fun(15.4, 0.2, 10.0)

13.357969920285026